In [1]:
import os
import os.path
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests, json
import re
from bs4 import BeautifulSoup, Tag
import uuid
import time
import warnings
warnings.filterwarnings("ignore")

# Part 1

The Webscraping script is to download clothes images(particularly) girl's Top along with some details such as type of sleeves, type of neck, type of top.

The script is in two Part,

Part 1: Create multiple CSV file which contain image urls and image captions. Each multiple CSV files then combine in one for preprocessing such as remove duplicate urls

Part 2: Browse individual urls from the CSV, then download and list all the image captions in text file

In [2]:
def get_next_20_items(counter):
    
    page_url = "https://www.snapdeal.com/acors/json/product/get/search/745/" + str(counter) + "/20?q=&sort=plrty"
    
    res = requests.get(url=page_url)

    item_list = []
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text,'html.parser')    
        product_tupple_images = soup.find_all('div',{'class':'product-tuple-image'})
        product_descs = soup.find_all('div',{'class':'product-desc-rating'})
        
        for product_image, prod_desc in zip(product_tupple_images,product_descs):
            
            item_detail = {}
            prod_link = product_image.find('a',{'class':'dp-widget-link'})
            item_detail['item_link'] = prod_link['href'].strip()
            
           
            prod_image_link = product_image.find('source',{'class':'product-image'})  
            item_detail['item_image_link'] = prod_image_link['srcset'].strip()
             
            
            prod_desc_link = prod_desc.find('a',{'class':'dp-widget-link noUdLine'})
            item_detail['prod_desc_link'] = prod_desc_link['href'].strip()
            
            prod_title = prod_desc.find('p',{'class':'product-title'})
            item_detail['prod_title'] = prod_title.text.strip()
            
            item_list.append(item_detail)   
    
    return item_list


In [3]:
def process_next_100_images(counter):
    
    min_range = (counter * 5) - 5
    max_range = (min_range + 5)
    
    total_products = []
    
    for indx in tqdm(range(min_range,max_range)):
        page_count_start = 20 * indx
        file_counter = 100 * counter
               
    
        item_list = get_next_20_items(page_count_start)
        
        for item in item_list:
            total_products.append(item) 
    
    return total_products

In [4]:
def generate_next_100_records(counter):
    total_products = process_next_100_images(counter)
    df = pd.DataFrame(data=total_products)
    unique_df = df[df.duplicated() == False]
    if unique_df.empty == False:
        counter_100 = counter * 100
        UNIQUE_URL_CSV_FILE = './snapdeal_data/csvfiles/unique_urls_' + str(counter_100) + ".csv"
        unique_df.to_csv(UNIQUE_URL_CSV_FILE,index=False)

### Create CSV file of unique_urls that contain product details and its image

In [6]:
SNAPDEAL_DATASET_DIR = './snapdeal_data/csvfiles/'

if not os.path.exists(SNAPDEAL_DATASET_DIR):
    os.makedirs(SNAPDEAL_DATASET_DIR)

for indx in range(1,25):
    generate_next_100_records(indx)

100%|██████████| 5/5 [00:02<00:00,  1.93it/s]


### Remove duplicate urls 

In [9]:
files = os.listdir('./snapdeal_data/csvfiles/')
files.sort(key=lambda f: int(re.sub('\D', '', f)))
full_df = []
for file_name in files:
    full_unique_urls_csv_path = './snapdeal_data/csvfiles/' + file_name
    print(full_unique_urls_csv_path)
    df = pd.read_csv(full_unique_urls_csv_path)
    print(df.shape)
    full_df.append(df)
    

result_df = pd.concat(full_df)

result_df = result_df[result_df.duplicated() == False]

SNAPDEAL_FILTERED_DATASET_URL_CSV = './snapdeal_data/filtered_csv/'

if not os.path.exists(SNAPDEAL_FILTERED_DATASET_URL_CSV):
    os.makedirs(SNAPDEAL_FILTERED_DATASET_URL_CSV)

result_df.to_csv('./snapdeal_data/filtered_csv/filtered_001.csv',index=False)

./snapdeal_data/csvfiles/unique_urls_100.csv
(78, 4)
./snapdeal_data/csvfiles/unique_urls_200.csv
(88, 4)
./snapdeal_data/csvfiles/unique_urls_300.csv
(94, 4)
./snapdeal_data/csvfiles/unique_urls_400.csv
(87, 4)
./snapdeal_data/csvfiles/unique_urls_500.csv
(96, 4)
./snapdeal_data/csvfiles/unique_urls_600.csv
(93, 4)
./snapdeal_data/csvfiles/unique_urls_700.csv
(95, 4)
./snapdeal_data/csvfiles/unique_urls_800.csv
(90, 4)
./snapdeal_data/csvfiles/unique_urls_900.csv
(91, 4)
./snapdeal_data/csvfiles/unique_urls_1000.csv
(99, 4)
./snapdeal_data/csvfiles/unique_urls_1100.csv
(100, 4)
./snapdeal_data/csvfiles/unique_urls_1200.csv
(82, 4)
./snapdeal_data/csvfiles/unique_urls_1300.csv
(99, 4)
./snapdeal_data/csvfiles/unique_urls_1400.csv
(98, 4)
./snapdeal_data/csvfiles/unique_urls_1500.csv
(69, 4)
./snapdeal_data/csvfiles/unique_urls_1600.csv
(96, 4)
./snapdeal_data/csvfiles/unique_urls_1700.csv
(83, 4)
./snapdeal_data/csvfiles/unique_urls_1800.csv
(99, 4)
./snapdeal_data/csvfiles/unique_urls

# Part 2

Download Each images from the unique url csv file and generate captions or tags from the url

In [10]:
def create_image_folder():
    image_filder_path = './snapdeal_data/tops/images/'
    caption_file = './snapdeal_data/tops/top_captions.txt'
    log_file = './snapdeal_data/tops/logfile.log' 
    
    if not os.path.exists(image_filder_path):
        os.makedirs(image_filder_path)
        
    file_caption = open(caption_file,'a+')        
    
    log_file = open(log_file,'a+')
    
    return image_filder_path, file_caption, log_file

In [11]:
def download_image(image_url, image_folder_path):
    img_data = requests.get(image_url).content
    myuuid = uuid.uuid4()
    extension = os.path.splitext(image_url)[1]
    file_name = str(myuuid) + extension         
    image_with_path = os.path.join(image_folder_path,file_name)
    
    with open(image_with_path, 'wb') as handler:
        handler.write(img_data)

    return file_name

In [12]:
def get_product_tags(product_desc_list):
    tag_list = ['Type:','Sleeves Length:','Neck Shape:','Sleeves Type:','Neck & Collar:']
    
    product_tag = {}
    product_processed_tag = ""
    
    for tag in tag_list:   
        ressults = [pd for pd in product_desc_list if pd.startswith(tag)]
        
        
        for result in ressults:
            if len(result) > 0:
                
                product_tag[tag[:-1]] = result.split(':')[1]
                tag_value = result.split(':')[1]
                tag_value = tag_value.replace(" ","")
                product_processed_tag = product_processed_tag + tag_value + "|"
    
    product_processed_tag = product_processed_tag[:-1]
    return product_tag, product_processed_tag

In [13]:
def access_product_page(prod_url):
    res = requests.get(url=prod_url)
    
    product_description_list = []
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text,'html.parser')    
        item_details = soup.find_all('li',{'class':'col-xs-8 dtls-li'})
        
        for item in item_details:
            item_val = item.find_all('span',{'class':'h-content'})
            item_val = item_val[0].get_text()
            product_description_list.append(item_val)       

    
    product_tags = get_product_tags(product_desc_list=product_description_list)
    return product_tags
    

In [14]:
image_file_path, caption_file, log_file = create_image_folder()

df = pd.read_csv('./snapdeal_data/filtered_csv/filtered_001.csv')

for item_link, item_image_link in tqdm(zip(df['item_link'],df['item_image_link'])):
    log_item = {}
    image_url = item_image_link
    product_image_file = download_image(image_url,image_file_path)
    
    item_url = item_link
    product_tags, image_processed_tags = access_product_page(prod_url=item_url)
    image_processed_tags = product_image_file +","+image_processed_tags      
    caption_file.writelines(image_processed_tags+"\n")
    
    log_item['item_link'] = item_url
    log_item['item_image_link'] = image_url
    log_item['product_all_tags'] = product_tags
    log_item['image_processed_tags'] = image_processed_tags
    log_item['image_file_location']= image_file_path + product_image_file 
    
    str_item = json.dumps(log_item)
    log_file.writelines(str_item + "\n")
    
caption_file.close()
log_file.close()

1292it [50:45,  2.36s/it]
